In [54]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from bokeh.io import output_notebook
output_notebook()
backtesting.set_bokeh_output(notebook=True)

Loading BokehJS ...

In [55]:
tsla_data = pd.read_csv("data/TSLA.csv")
tsla_data["Date"] = pd.to_datetime(tsla_data["Date"])
tsla_data = tsla_data[(tsla_data["Date"]>= '01/01/2022') & (tsla_data["Date"]>'01/01/2023')]
tsla_data.set_index("Date", inplace = True)
# tsla_data = pd.Series(tsla_data["Adj Close"].values, index = tsla_data["Date"])

In [56]:
class MA_crossover_bt(Strategy):
    short_period = 10
    long_period = 50

    def init(self):
        # Inicializando as médias móveis: curta e longa
        self.short_ma = self.I(self.moving_average, self.data.Close, self.short_period)
        self.long_ma = self.I(self.moving_average, self.data.Close, self.long_period)

    def moving_average(self, data, size):
        """Função para calcular a média móvel exponencial dos dados"""
        return pd.Series(data).ewm(span=size, adjust=False).mean()

    def next(self):
        if crossover(self.short_ma, self.long_ma):
            self.position.close()
            self.buy()
        elif crossover(self.long_ma, self.short_ma):
            self.position.close()
            self.sell()

In [57]:
# Parâmetros da estratégia
short_period = 10
long_period = 50

# Executando o backtest
bt = Backtest(tsla_data, MA_crossover_bt, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

Start                     2023-01-03 00:00:00
End                       2024-08-21 00:00:00
Duration                    596 days 00:00:00
Exposure Time [%]                   81.508516
Equity Final [$]                  5406.898295
Equity Peak [$]                  13635.439774
Return [%]                         -45.931017
Buy & Hold Return [%]              106.540248
Return (Ann.) [%]                  -31.410094
Volatility (Ann.) [%]               30.435927
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -60.358461
Avg. Drawdown [%]                  -19.550368
Max. Drawdown Duration      400 days 00:00:00
Avg. Drawdown Duration       95 days 00:00:00
# Trades                                   16
Win Rate [%]                            18.75
Best Trade [%]                      22.459725
Worst Trade [%]                    -16.534617
Avg. Trade [%]                    

In [58]:
stats.iloc[0]

Timestamp('2023-01-03 00:00:00')

In [59]:
bt.plot()


C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p3276', ...)

In [60]:
class MACD(Strategy):
    """ ADD stop loss"""
    short_period = 12
    long_period = 26   
    signal_period = 9 
    

    def init(self):
        # Calculate short and long moving averages
        self.short_ma = self.I(self.moving_average, self.data.Close, self.short_period)
        self.long_ma = self.I(self.moving_average, self.data.Close, self.long_period)
        
        # MACD line: difference between short and long moving averages
        self.macd_line = self.short_ma - self.long_ma
        
        # Signal line: EMA of the MACD line
        self.signal_line = self.I(self.moving_average, self.macd_line, self.signal_period)

    def moving_average(self, data, size):
        """Exponential moving average"""
        return pd.Series(data).ewm(span=size, adjust=False).mean()

    def next(self):
        # Buy when MACD line crosses above the signal line
        if crossover(self.macd_line, self.signal_line):
            if self.position.is_short:
                self.position.close()  # Close any short positions before buying
            self.buy()

        # Sell when MACD line crosses below the signal line
        elif crossover(self.signal_line, self.macd_line):
            if self.position.is_long:
                self.position.close()  # Close any long positions before selling
            self.sell()

In [61]:
# Executando o backtest
bt = Backtest(tsla_data, MACD, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

Start                     2023-01-03 00:00:00
End                       2024-08-21 00:00:00
Duration                    596 days 00:00:00
Exposure Time [%]                   91.727494
Equity Final [$]                 24351.683074
Equity Peak [$]                  27672.734767
Return [%]                         143.516831
Buy & Hold Return [%]              106.540248
Return (Ann.) [%]                   72.582149
Volatility (Ann.) [%]               87.264191
Sharpe Ratio                         0.831752
Sortino Ratio                         2.44232
Calmar Ratio                         2.611953
Max. Drawdown [%]                   -27.78846
Avg. Drawdown [%]                   -6.623716
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                   26
Win Rate [%]                        57.692308
Best Trade [%]                      40.018423
Worst Trade [%]                    -14.327409
Avg. Trade [%]                    

In [62]:
bt.plot()


C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p3692', ...)

# Boillinger Bands

In [63]:
import numpy as np

In [64]:

class BollingerBands(Strategy):
    period = 20  # Standard period for Bollinger Bands
    std_dev = 2  # Number of standard deviations for the bands
    stop_loss_pct = 0.05  # Stop loss at 5%

    def init(self):
        # Compute moving average (SMA)
        self.ma = self.I(self.moving_average, self.data.Close, self.period)

        # Compute rolling standard deviation (compatible with Backtesting.py)
        self.std = self.I(self.rolling_std, self.data.Close, self.period)

        # Compute upper and lower Bollinger Bands
        self.upper_band = self.ma + self.std * self.std_dev
        self.lower_band = self.ma - self.std * self.std_dev

    def moving_average(self, data, period):
        """Simple moving average (SMA)"""
        return pd.Series(data).ewm(span=period, adjust=False).mean()


    def rolling_std(self, data, period):
        """Rolling standard deviation"""
        return np.array([np.std(data[i - period:i]) for i in range(period, len(data) + 1)])

    def next(self):
        # If the price crosses below the lower band, we buy
        if crossover(self.lower_band, self.data.Close):
            if self.position.is_short:
                self.position.close()  # Close any short positions before buying
            self.buy()

        # If the price crosses above the upper band, we sell
        elif crossover(self.data.Close, self.upper_band):
            if self.position.is_long:
                self.position.close()  # Close any long positions before selling
            self.sell()

        # Implement stop loss
        if self.position:
            if self.position.is_long and self.data.Close[-1] < self.position.entry_price * (1 - self.stop_loss_pct):
                self.position.close()
            elif self.position.is_short and self.data.Close[-1] > self.position.entry_price * (1 + self.stop_loss_pct):
                self.position.close()

In [65]:
# Executando o backtest
bt = Backtest(tsla_data, BollingerBands, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

ValueError: Indicators must return (optionally a tuple of) numpy.arrays of same length as `data` (data shape: (411,); indicator "rolling_s…(C,20)"shape: (392,), returned value: [20.59008074 22.08899232 23.91058971 24.98174404 25.99032961 26.96562319
 27.64449435 28.47375711 28.02634745 26.86943179 26.76918537 26.18301421
 24.01353628 22.05816995 20.07983681 17.58459165 14.23582707 12.07669137
 11.70328422  9.63815705  7.73459304  6.82136804  6.26807218  5.98435646
  6.52838523  7.66947475  9.66012744 10.8695451  11.91377508 12.22057446
 12.17358634 11.38027575 11.43728921 10.84394701 10.85129572 10.56294399
 10.18368064 10.02693744  9.07201065  8.10374485  7.44167335  7.62927218
  8.5632054   8.60256584  8.67997985  8.601612    7.94045335  7.24008969
  6.47456582  6.63030601  6.36625701  6.33155572  5.98141146  5.92583102
  6.00277924  8.25895692  9.5489088  10.81560092 11.91486316 13.46873821
 13.9402734  13.88013223 12.62236851 12.31805029 11.93334439 11.82170067
 11.41675774 10.9877